In [2]:
!pip install PyQt5

  Obtaining dependency information for PyQt5 from https://files.pythonhosted.org/packages/ca/ac/596e8ca16fd0634542d874c0d79219fc527ea7de73a5000092f60ecbf6e9/PyQt5-5.15.10-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for PyQt5-sip<13,>=12.13 from https://files.pythonhosted.org/packages/8e/e2/d296cb17bae19ba49137a2649934a70d5f48cc99b7daa6ce9cf1aecddfeb/PyQt5_sip-12.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl (50.1 MB)
Using cached PyQt5-5.15.10-cp37-abi3-win_amd64.whl (6.8 MB)
Using cached PyQt5_sip-12.13.0-cp311-cp311-win_amd64.whl (78 kB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib qt
import seaborn as sns
import os
from os import path
import datetime
import xlrd
plt.style.use('seaborn-v0_8-ticks')
from scipy.signal import find_peaks
import itertools
from pylab import *
import csv

Pobieranie zestawu danych

In [ ]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing"
folders = os.listdir(folder_path)
breathing_dict = {folder : {} for folder in folders}

for folder_name in folders:
    path = os.path.join(folder_path, folder_name)
    for file_name in os.listdir(path):
        if file_name.endswith('.csv'):
         df_name = file_name.split('_ODD')[0]
         df = pd.read_csv(os.path.join(path, file_name))
         breathing_dict[folder_name][df_name] = df

Pobranie zestawu danych, gdzie występują dwa typy plików csv

In [3]:
collective_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing_full2"
folders = os.listdir(collective_path)
breathing_dict = {folder : {} for folder in folders}

for folder_name in folders:
    folder_path = os.path.join(collective_path, folder_name)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
          df_name = file_name.split('PAC_')[1].split('_')[0]
          file_path = os.path.join(folder_path, file_name)
          csv_file = open(file_path, 'r')
          try:
              dialect = csv.Sniffer().sniff(csv_file.readline(), ';')   
          except:
              dialect = csv.Sniffer().sniff(csv_file.readline(), ',')   
          csv_file.seek(0)
          df = pd.read_csv(file_path, on_bad_lines='skip', sep=dialect.delimiter)
          if dialect.delimiter == ";":
              for row in df:
                  df[row] = df[row].str.replace(',','.').astype(float)
          breathing_dict[folder_name][df_name] = df

In [4]:
breathing_dict['6_breaths']['48']

,DateTime,fv_r[],fv_l[],abp_finger[mm_Hg],abp_arm[mm_Hg],ekg[],marker[],etco2[mm_Hg],rr[bpm],co2[mm_Hg]
0,42165.618457,44.71,75.16,91.64,107.90,-0.1880,0.0119,41.0,13.0,38.0
1,42165.618457,44.78,75.26,89.05,105.70,-0.1620,-0.0006,41.0,13.0,38.0
2,42165.618457,39.54,71.91,88.56,105.60,-0.1743,0.0156,41.0,13.0,38.0
3,42165.618457,39.48,72.01,86.88,103.40,-0.1642,-0.0046,41.0,13.0,38.0
4,42165.618457,39.48,71.97,85.69,101.70,-0.1587,0.0113,41.0,13.0,38.0
...,...,...,...,...,...,...,...,...,...,...
65125,42165.622357,56.43,121.90,124.00,93.32,0.2222,0.0027,36.0,12.0,11.0
65126,42165.622357,56.26,121.90,125.60,90.85,0.1364,0.0137,36.0,12.0,11.0
65127,42165.622357,56.33,121.90,126.10,89.97,0.0922,0.0116,36.0,12.0,11.0
65128,42165.622357,55.32,123.00,126.10,89.63,0.0531,-0.0085,36.0,12.0,11.0


Sprawdzenie nazw kolumn w plikach

In [5]:
for breath_freq in breathing_dict:
    for df_name, df in breathing_dict[breath_freq].items(): 
        print(df.columns)

Index(['DateTime', 'fv_r[fv_r]', 'fv_l[fv_l]', 'abp_finger[abp_finger]',
       'abp_arm[abp_arm]', 'ekg[ekg]', 'marker[marker]', 'etco2[etco2]',
       'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r[fv_r]', 'fv_l[fv_l]', 'abp_finger[abp_finger]',
       'abp_arm[abp_arm]', 'ekg[ekg]', 'marker[marker]', 'etco2[etco2]',
       'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r[fv_r]', 'fv_l[fv_l]', 'abp_finger[abp_finger]',
       'abp_arm[abp_arm]', 'ekg[ekg]', 'marker[marker]', 'etco2[etco2]',
       'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r[fv_r]', 'fv_l[fv_l]', 'abp_finger[abp_finger]',
       'abp_arm[abp_arm]', 'ekg[ekg]', 'marker[marker]', 'etco2[etco2]',
       'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r[fv_r]', 'fv_l[fv_l]', 'abp_finger[abp_finger]',
       'abp_arm[abp_arm]', 'ekg[ekg]', 'marker[marker]', 'etco2[etco2]',
       'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['Da

Zamiana nazw kolumn na jednolite
"fv_r[]", "fv_r[fv_r]" -> "fv_r"
"fv_l[]", "fv_r[fv_l]" -> "fv_l"
"ekg[]", "ekg[ekg]" -> "ekg"
"abp_arm[mm_Hg]", "abp_arm[abp_arm]" -> "abp_arm"

In [6]:
column_names_map = {"fv_r[]":"fv_r", "fv_r[fv_r]":"fv_r", "fv_l[]":"fv_l", 
                    "fv_l[fv_l]":"fv_l", "ekg[]":"ekg", "ekg[ekg]":"ekg", "abp_arm[mm_Hg]":"abp_arm", "abp_arm[abp_arm]":"abp_arm"}
for breath_freq in breathing_dict:
    for df_name, df in breathing_dict[breath_freq].items(): 
        df.rename(columns=column_names_map, inplace=True)

Sprawdzenie nazw kolumn w plikach po raz drugi

In [7]:
for breath_freq in breathing_dict:
    for df_name, df in breathing_dict[breath_freq].items(): 
        print(df.columns)

Index(['DateTime', 'fv_r', 'fv_l', 'abp_finger[abp_finger]', 'abp_arm', 'ekg',
       'marker[marker]', 'etco2[etco2]', 'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r', 'fv_l', 'abp_finger[abp_finger]', 'abp_arm', 'ekg',
       'marker[marker]', 'etco2[etco2]', 'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r', 'fv_l', 'abp_finger[abp_finger]', 'abp_arm', 'ekg',
       'marker[marker]', 'etco2[etco2]', 'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r', 'fv_l', 'abp_finger[abp_finger]', 'abp_arm', 'ekg',
       'marker[marker]', 'etco2[etco2]', 'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r', 'fv_l', 'abp_finger[abp_finger]', 'abp_arm', 'ekg',
       'marker[marker]', 'etco2[etco2]', 'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['DateTime', 'fv_r', 'fv_l', 'abp_finger[abp_finger]', 'abp_arm', 'ekg',
       'marker[marker]', 'etco2[etco2]', 'rr[rr]', 'co2[co2]'],
      dtype='object')
Index(['Da

Funkcja konwertująca czas

In [8]:
def icmp_dateformat_to_datetime(icmp_time_mark):
    datetime_date = xlrd.xldate_as_datetime(icmp_time_mark, 0)
    datetime_date = datetime_date + datetime.timedelta(hours=1)
    return datetime_date

Sprawdzenie procentu brakujących wartości w kolumnach z ecg, abp oraz fv

In [9]:
nan_dict = {breath_freq : {} for breath_freq in breathing_dict}
for breath_freq in breathing_dict:
    for df_name, df in breathing_dict[breath_freq].items():
        nan_dict[breath_freq][df_name] = {}
        for col in ['abp_arm', 'ekg', 'fv_r', 'fv_l']:
           nan_percent = df[col].isnull().sum() * 100 / len(df)
           if nan_percent != 0.0:
               nan_dict[breath_freq][df_name][col] = round(nan_percent, 2)
nan_df = pd.DataFrame(nan_dict).T
nan_df

,10,11,12,13,14,15,16,17,18,19,...,52,53,54,55,56,5,6,7,8,9
10_breaths,{},{'abp_arm': 0.79},"{'abp_arm': 0.91, 'ekg': 0.08, 'fv_r': 0.08, '...",{'abp_arm': 1.12},{},{},{'abp_arm': 1.03},{},{},{},...,{},{},{},"{'abp_arm': 0.2, 'ekg': 0.2, 'fv_r': 0.2, 'fv_...",{},{},{},{},{},{}
15_breaths,"{'abp_arm': 1.52, 'ekg': 0.03, 'fv_r': 38.61, ...",{},"{'fv_r': 0.33, 'fv_l': 0.12}",{},"{'abp_arm': 0.11, 'ekg': 0.11, 'fv_r': 0.11, '...",{},{},{'abp_arm': 1.36},{},{},...,{},{},{},{},{},{},{},{},{},{}
6_breaths,{},{},{'fv_r': 0.13},{},{},{'fv_r': 0.29},{},{},{},{},...,{},{},"{'abp_arm': 0.07, 'ekg': 0.07, 'fv_r': 0.07, '...",{},"{'abp_arm': 0.04, 'ekg': 0.04, 'fv_r': 0.04, '...",{'abp_arm': 2.12},{},{'ekg': 0.0},{},{}
baseline,{'abp_arm': 3.23},{},{'abp_arm': 1.77},{'abp_arm': 0.79},{'abp_arm': 2.44},{'abp_arm': 0.58},{'abp_arm': 1.1},{'abp_arm': 1.91},"{'abp_arm': 0.16, 'ekg': 0.16, 'fv_r': 0.16, '...",{'abp_arm': 0.85},...,"{'abp_arm': 0.11, 'ekg': 0.11, 'fv_r': 0.11, '...","{'abp_arm': 0.12, 'ekg': 0.12, 'fv_r': 0.12, '...",{},{},"{'abp_arm': 0.02, 'ekg': 0.02, 'fv_r': 0.02, '...",{'ekg': 0.0},"{'abp_arm': 3.29, 'fv_r': 0.43, 'fv_l': 0.47}",{'abp_arm': 0.71},{'fv_r': 53.27},"{'abp_arm': 0.22, 'ekg': 0.22, 'fv_r': 0.22, '..."


Dodanie kolumny z datą i godziną do każdego dataframe i wypełnienie braków danych (NaN) wartościami średnimi

In [10]:
for breath_freq in breathing_dict:
    for df_name, df in breathing_dict[breath_freq].items():
               df['DateFormat'] = df['DateTime'].map(icmp_dateformat_to_datetime)
               df.fillna(df.mean(), inplace=True)

Funkcja znajdująca piki, diff to jest RR w przypadku ECG tylko trzeba je przemnożyć przez 0.005 (1/(200 Hz)), peaks to numery próbek odpowiadających pikom, signal[peaks] to wartości sygnału odpowiadające tym numerom próbek

In [11]:
def get_peaks(signal):
    dist = 115
    peaks, _ = find_peaks(signal, distance=dist)
    difference = np.diff(peaks)  
    return peaks, signal[peaks], difference

Wyrysowanie 600 pierwszych próbek każdego sygnału z zaznaczonymi znalezionymi pikami ("x")

In [15]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\timeseries_plots2"
for breath_freq in breathing_dict:
    for df_name, df in breathing_dict[breath_freq].items():       
        fig, ax = plt.subplots(4, 1, tight_layout = True, figsize=(16, 32)) 
        for i, col in enumerate(['abp_arm', 'ekg', 'fv_l', 'fv_r']):
           x = df[col].iloc[:600]
           ax[i].plot(x)
           my_peaks = get_peaks(x)
           ax[i].plot(my_peaks[0], my_peaks[1], "x")
           ax[i].set_ylabel(f'{col}', fontsize=25)
           ax[i].tick_params(axis='both', labelsize=18)
           ax[i].grid(True)
        ax[3].set_xlabel("nr probki", fontsize = 28)
        path = folder_path+f"\\{breath_freq}\\{df_name}_plot.pdf"
        plt.savefig(path, bbox_inches='tight', dpi=600)
        plt.show()
        plt.close()

In [54]:
N_PATIENTS = 15

In [77]:
def make_empty_matrix():
    MY_PERMUTATIONS = [''.join(map(str, item)) for item in list((itertools.product([1, 0], repeat=3)))]
    my_matrix = pd.DataFrame()
    for col1 in MY_PERMUTATIONS: 
       for col2 in MY_PERMUTATIONS:
            my_matrix.at["'"+col1, "'"+col2] = 0
    return my_matrix

In [57]:
def get_diff_df(dataframe, is_fv, is_left):
    sap = get_peaks(dataframe['abp_arm'])[1]
    if is_fv:
        sap = get_peaks(dataframe['fv_l'])[1]
        if not is_left:
            sap = get_peaks(dataframe['fv_r'])[1]
    rr = get_peaks(dataframe['ekg'])[2]*0.005
    sap_diff = np.diff(sap)
    rr_diff = np.diff(rr)
    diff_dataframe = pd.DataFrame(list(zip(sap_diff, rr_diff)), columns=['sap_diff', 'rr_diff'])
    return diff_dataframe

In [64]:
def make_and_save_heatmap(relative_array, breath_frequency, is_fv):
    my_custom_palette = sns.color_palette("ch:-.24", as_cmap=True)
    heatmap_path = f"C:\\Users\\48503\\Desktop\\joint_symbolic\\heatmaps\\"
    my_heatmap = sns.heatmap(relative_array,cmap = my_custom_palette, annot=True)
    my_heatmap.xaxis.tick_top()
    my_heatmap.xaxis.set_ticks_position('none')
    fontsize = 10
    plt.xlabel('SAP', fontsize = fontsize) 
    my_heatmap.xaxis.set_label_position('top') 
    plt.ylabel('RR', fontsize = fontsize) 
    if is_fv:
        plt.savefig(heatmap_path+f"new_{breath_frequency}_heatmap_fv.pdf", bbox_inches='tight', dpi=600)
        relative_array.to_csv(heatmap_path+f"new_{breath_frequency}_matrix_fv.csv")
    else:
        plt.savefig(heatmap_path+f"new_{breath_frequency}_heatmap.pdf", bbox_inches='tight', dpi=600)
        relative_array.to_csv(heatmap_path+f"new_{breath_frequency}_matrix.csv")
    plt.show()
    plt.close()

In [78]:
def joint_symbolic(data_dict, is_fv):
    jsd_sym_dict, jsd_diam_dict = {freq:{} for freq in data_dict},{freq:{} for freq in data_dict}
    metadata = pd.read_csv("C:\\Users\\48503\\Desktop\\joint_symbolic\\dane_sonata\\dane_sonata.csv", sep = ";")
    for freq in data_dict:
        freq_matrix = make_empty_matrix()
        for patient_number, dataframe in data_dict[freq].items():
            is_left = True
            if is_fv:
                try:
                    is_left = bool(metadata.at[metadata.loc[metadata['L.poj.'] == patient_number].index[0], 'FV_LEWA_MCA'])
                except:
                    is_left = True
            
            patient_freq_matrix = make_empty_matrix()
            diff_df = get_diff_df(dataframe, is_fv, is_left)
            
            for col_name in ['sap_diff', 'rr_diff']:
                diff_df[f'{col_name}_binary'] = diff_df.apply(lambda df_row: 1 if df_row[col_name] > 0 else 0, axis = 1)
            
            for index in range(0, len(diff_df)):
                binary_sap =  ''.join(diff_df['sap_diff_binary'].iloc[index:index+3].astype(str))
                binary_rr =  ''.join(diff_df['rr_diff_binary'].iloc[index:index+3].astype(str))
                if len(binary_rr) > 2 and len(binary_sap) > 2:
                     freq_matrix.at["'"+binary_rr, "'"+binary_sap] += 1
                     patient_freq_matrix.at["'"+binary_rr, "'"+binary_sap] += 1 
            
            relative_patient_matrix = patient_freq_matrix.div(np.sum(patient_freq_matrix.values))   
            jsd_sym_dict[freq][patient_number] = np.sum(np.diag(relative_patient_matrix))
            jsd_diam_dict[freq][patient_number] = np.sum(np.diag(np.rot90(relative_patient_matrix)))
            
        relative_freq_matrix = freq_matrix.div(np.sum(freq_matrix.values))        
        make_and_save_heatmap(relative_freq_matrix, freq, is_fv)
        
    jsd_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\jsd\\"
    if is_fv:
        pd.DataFrame(jsd_sym_dict).to_csv(jsd_path+"new_jsd_sym_fv.csv")
        pd.DataFrame(jsd_diam_dict).to_csv(jsd_path+"new_jsd_diam_fv.csv")
    else:    
        pd.DataFrame(jsd_sym_dict).to_csv(jsd_path+"new_jsd_sym.csv")
        pd.DataFrame(jsd_diam_dict).to_csv(jsd_path+"new_jsd_diam.csv")

rr-sap

In [79]:
joint_symbolic(breathing_dict, False)

fv

In [80]:
joint_symbolic(breathing_dict, True)

In [101]:
path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\heatmaps"
for file_name in os.listdir(path):
   if file_name.endswith('.csv'):
      df = pd.read_csv(path+f"\\{file_name}", index_col='Unnamed: 0')
      x_labels = list(df.columns)
      y_labels = list(df.columns)
      x, y = np.meshgrid(range(len(x_labels)), range(len(y_labels)))
      Z = df.values
      fig = plt.figure()
      ax = fig.add_subplot(111, projection='3d')
      ax.bar3d(x.ravel(), y.ravel(), 0, 1, 1, Z.ravel(), shade=False)
      ax.view_init(elev=30, azim=-60)
      ax.set_xticks(range(len(x_labels)))
      ax.set_xlim(0, len(x_labels))
      ax.set_xticklabels(x_labels) 
      ax.set_yticks(range(1, len(y_labels)+1))
      ax.set_yticklabels(y_labels) 
      ax.set_xlabel('RR')
      ax.set_ylabel('SAP')
      ax.set_zlabel('czestotliwosc')
      plt.savefig(path+f"\\{file_name.split('_matrix')[0]}_bar3d.pdf", bbox_inches='tight', dpi=600)
      plt.show()
      plt.close()

In [87]:
path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\heatmaps"
save_path = path+"\\transformed"
for file_name in os.listdir(path):
   if file_name.endswith('.csv'):
      df = pd.read_csv(path+f"\\{file_name}", index_col='Unnamed: 0')
      new_df = pd.DataFrame(columns=['RR', 'SAP', 'value'])

      for idx in df.index:
         for col in df.columns:
            new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'value': [df.at[idx, col]]})], ignore_index=True)
      new_df.to_csv(save_path+"\\"+file_name, index=False)

C:\Users\48503\AppData\Local\Temp\ipykernel_29284\3704639540.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'value': [df.at[idx, col]]})], ignore_index=True)
C:\Users\48503\AppData\Local\Temp\ipykernel_29284\3704639540.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'value': [df.at[idx, col]]})], ignore_index=True)
C:\Users\48503\AppData\Local\Temp\